In [54]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
from statsmodels.tsa.arima.model import ARIMA
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from joblib import Parallel, delayed
from scipy.stats import norm
import time
from itertools import product
from tqdm import tqdm
from pandas.tseries.offsets import BDay

In [32]:
df = pd.read_csv('data/DJIA_20140831_20250220.csv', parse_dates=['TradingDate'])
df.head()   

,TradingDate,Symbol,StatusID,Filling,OpenPrice,HighPrice,LowPrice,ClosePrice,AvgPrice,Volume,Amount,TurnoverRate
0,2021-02-22,IBM,0,0,118.50,121.125,118.44,120.86,120.2135,5879249.0,7.067648e+08,0.660
1,2021-02-22,MMM,0,0,176.46,177.090,175.01,176.12,175.8778,2271502.0,3.995067e+08,0.392
2,2021-02-22,WMT,0,0,137.74,138.340,136.25,137.69,137.2495,10702210.0,1.468873e+09,0.378
3,2021-02-22,DIS,0,0,181.74,194.020,181.53,191.76,190.6355,18799609.0,3.583873e+09,1.036
4,2021-02-22,WBA,0,0,48.75,49.170,48.40,48.86,48.8086,4379740.0,2.137691e+08,0.507


In [33]:
df.set_index('TradingDate', inplace=True)
df.sort_index(inplace=True)
df.head()

,Symbol,StatusID,Filling,OpenPrice,HighPrice,LowPrice,ClosePrice,AvgPrice,Volume,Amount,TurnoverRate
TradingDate,,,,,,,,,,,
2014-09-02,AAPL,0,0,103.06,103.74,102.72,103.30,103.30,53564262.0,5.533188e+09,0.895
2014-09-02,CRM,0,0,59.20,60.11,59.11,59.85,59.85,5268153.0,3.152990e+08,0.851
2014-09-02,AMGN,0,0,139.62,139.93,137.20,137.96,137.96,3290710.0,4.539864e+08,0.433
2014-09-02,AXP,0,0,89.90,90.13,89.28,89.72,89.72,3592108.0,3.222839e+08,0.343
2014-09-02,NKE,0,0,78.35,79.52,78.35,79.28,79.28,4082183.0,3.236355e+08,0.591


In [34]:
df = df.rename(columns={'ClosePrice': 'close', 'OpenPrice': 'open', 'HighPrice': 'high', 'LowPrice': 'low', 'Amount': 'amount', 'Volume': 'volume', 'TurnoverRate': 'turnover'})

In [35]:
df.drop(['StatusID', 'Filling'], axis=1, inplace=True)

In [36]:
components_20200831_20240206 =  ['HON', 'NKE', 'CVX', 'GS', 'JNJ', 'AAPL', 'HD', 'MSFT', 'VZ', 'CSCO', 'AXP', 'AMGN', 'MRK', 'JPM', 'WMT', 'CRM', 'V', 'CAT', 'DIS', 'MMM',
'BA', 'PG', 'KO','UNH', 'MCD', 'TRV', 'WBA', 'IBM', 'INTC', 'DOW']

In [37]:
# Function to determine the best ARIMA order and keep a rolling model copy
# Intermediate function
def best_arima(df, max_p=5, max_q=5, max_d=3):
    """
    Automatically selects the best ARIMA hyperparameters (p, d, q) based on AIC and fits the model.
    Returns both the best order and the fitted model (model_fit).
    If no valid model is found, returns (None, None).
    """
    series = df['close']
    exog =df[['open', 'high', 'low', 'amount', 'volume', 'turnover']]

    def _try_arima(p, d, q, series, exog):
        try:
            # Suppress warnings during model fitting
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = ARIMA(endog=series, exog=exog, order=(p, d, q))
                model_fit = model.fit()
            return (p, d, q), model_fit, model_fit.aic
        except:
            return None, None, float('inf')
    
    # Parallel processing of ARIMA models
    param_combinations = list(product(range(1,max_p), range(1, max_d), range(1, max_q)))
    results = Parallel(n_jobs=-1)(
        delayed(_try_arima)(p, d, q, series, exog) 
        for p, d, q in param_combinations
    )
    
    # Find the best model
    best_aic = float('inf')
    best_order = None
    best_model_fit = None
    
    for order, model_fit, aic in results:
        if aic < best_aic and order is not None:
            best_aic = aic
            best_order = order
            best_model_fit = model_fit
    
    return best_order, best_model_fit
        

In [46]:
df.head()

,Symbol,open,high,low,close,AvgPrice,volume,amount,turnover
TradingDate,,,,,,,,,
2014-09-02,AAPL,103.06,103.74,102.72,103.30,103.30,53564262.0,5.533188e+09,0.895
2014-09-02,CRM,59.20,60.11,59.11,59.85,59.85,5268153.0,3.152990e+08,0.851
2014-09-02,AMGN,139.62,139.93,137.20,137.96,137.96,3290710.0,4.539864e+08,0.433
2014-09-02,AXP,89.90,90.13,89.28,89.72,89.72,3592108.0,3.222839e+08,0.343
2014-09-02,NKE,78.35,79.52,78.35,79.28,79.28,4082183.0,3.236355e+08,0.591


In [ ]:
start = '2016-02-01'
end = '2025-01-02'
forecast_steps = 6 # 第6天
data_all = {}

for component in components_20200831_20240206:

    data = df[df['Symbol'] == component].copy()
    data["ARIMAX_Forecast"] = np.nan
    data["ARIMAX_Residuals"] = np.nan

    # data.sort_index(inplace=True)
    data.head()

    for cur_date in data.loc[start:end].index:
        print("Current date:", cur_date)

        cur_idx = data.index.get_loc(cur_date)
        new_date = data.index[cur_idx + forecast_steps]
        print("New date:", new_date)

        training_data = data.loc[:cur_date]

        # Fit ARIMAX model
        order, model_fit = best_arima(training_data)
        if model_fit is not None:
            exog_cols = ['open', 'high', 'low', 'amount', 'volume', 'turnover']
            # Note: .loc[cur_date, exog_cols] returns a Series. We then reshape it.
            exog_current = training_data.loc[cur_date, exog_cols].astype(float)

            # Repeat the row forecast_steps times to create a DataFrame for forecast.
            exog_forecast = pd.DataFrame(
            np.tile(exog_current.values, (forecast_steps, 1)),
                columns=exog_cols
        ).astype(float)

            forecasts = model_fit.forecast(steps=forecast_steps, exog=exog_forecast)
            forecast_value = forecasts.iloc[-1]

            data.loc[cur_date, 'ARIMAX_Forecast'] = forecast_value
            data.loc[cur_date, 'ARIMAX_Residuals'] = data.loc[new_date, 'close'] - forecast_value

            print(f"Forecasted value for {component} on {new_date}: {forecast_value}")
            print(f"Residuals for {component} on {new_date}: {data.loc[new_date, 'close'] - forecast_value}")
            print()
            
    data_all[component] = data     

Current date: 2016-02-01 00:00:00
New date: 2016-02-09 00:00:00


c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Forecasted value for HON on 2016-02-09 00:00:00: 102.4087673471879
Residuals for HON on 2016-02-09 00:00:00: 0.31123265281209456

Current date: 2016-02-02 00:00:00
New date: 2016-02-10 00:00:00


c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Forecasted value for HON on 2016-02-10 00:00:00: 100.78088686519634
Residuals for HON on 2016-02-10 00:00:00: 1.8191131348036578

Current date: 2016-02-03 00:00:00
New date: 2016-02-11 00:00:00


...

Forecasted value for HON on 2020-03-30 00:00:00: 114.7874750939534
Residuals for HON on 2020-03-30 00:00:00: 16.962524906046596

Current date: 2020-03-23 00:00:00
New date: 2020-03-31 00:00:00


c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Forecasted value for HON on 2020-03-31 00:00:00: 105.9070208997495
Residuals for HON on 2020-03-31 00:00:00: 27.882979100250495

Current date: 2020-03-24 00:00:00
New date: 2020-04-01 00:00:00


c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Forecasted value for HON on 2020-04-01 00:00:00: 118.76519374488936
Residuals for HON on 2020-04-01 00:00:00: 10.944806255110649

Current date: 2020-03-25 00:00:00
New date: 2020-04-02 00:00:00


In [ ]:

# ----------------------------
# 1. Load and Prepare Data
# ----------------------------
# Assume your CSV file has columns: Date, Open, High, Low, Close, Volume, Turnover
df = pd.read_csv('your_stock_data.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

# ----------------------------
# 2. Stationarity Check (Optional)
# ----------------------------
# Check if Close is stationary. If not, you might consider differencing.
def adf_test(series, title=''):
    print(f'ADF Test: {title}')
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    for key, value in result[4].items():
        print(f'Critical Value {key}: {value:.4f}')
    print("\n")
    return result[1]

p_val = adf_test(df['Close'], title='Close Price')
if p_val > 0.05:
    print("Series is non-stationary; consider differencing or detrending for ARIMAX.")
    # Optionally, uncomment the next two lines to difference the series:
    # df['Close'] = df['Close'].diff()
    # df.dropna(inplace=True)

# ----------------------------
# 3. Fit ARIMAX Model
# ----------------------------
# Use the exogenous variables from the dataset.
exog_vars = ['Open', 'High', 'Low', 'Volume', 'Turnover']
exog = df[exog_vars]

# Set ARIMAX order. Here (2,1,2) is an example; adjust based on your analysis.
order = (2, 1, 2)
arimax_model = ARIMA(endog=df['Close'], exog=exog, order=order)
arimax_fit = arimax_model.fit()
print(arimax_fit.summary())

# In-sample fitted values and residuals:
df['ARIMAX_Fitted'] = arimax_fit.fittedvalues
df['ARIMAX_Residuals'] = df['Close'] - df['ARIMAX_Fitted']

# ----------------------------
# 4. Forecast with ARIMAX for Next 6 Days
# ----------------------------
forecast_steps = 6

# In practice, you need to forecast or obtain exogenous values for the forecast horizon.
# Here we assume the exogenous values remain constant at the last observed values.
last_exog = exog.iloc[-1].values.reshape(1, -1)
exog_forecast = pd.DataFrame(np.repeat(last_exog, forecast_steps, axis=0), 
                             columns=exog.columns)

# ARIMAX forecast:
arimax_forecast = arimax_fit.forecast(steps=forecast_steps, exog=exog_forecast)
print("ARIMAX Forecast (Linear Component):")
print(arimax_forecast)

# ----------------------------
# 5. AutoML for Predicting ARIMAX Residuals
# ----------------------------
# Prepare training data for residual prediction using the same exogenous variables.
# (Optionally, you can add lags or other features.)
X_res = exog.copy()
y_res = df['ARIMAX_Residuals']

# Drop missing values if any:
X_res = X_res.dropna()
y_res = y_res.loc[X_res.index]

# Split the training data (here we use all data for simplicity; for production, use cross-validation)
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2, random_state=42)
tpot.fit(X_res, y_res)

# Forecast ARIMAX residuals for the next 6 days using the same exogenous forecast values.
ml_residual_forecast = tpot.predict(exog_forecast)
print("TPOT Forecast for ARIMAX Residuals:")
print(ml_residual_forecast)

# ----------------------------
# 6. Combine Forecasts
# ----------------------------
final_forecast = arimax_forecast + ml_residual_forecast
print("Final Combined Forecast (ARIMAX + ML Residual):")
print(final_forecast)

# ----------------------------
# 7. Visualization
# ----------------------------
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Close'], label='Historical Close Price')
# Plot ARIMAX fitted values
plt.plot(df.index, df['ARIMAX_Fitted'], label='ARIMAX Fitted', alpha=0.7)
# Create forecast index (using business days or a custom frequency as needed)
forecast_index = pd.date_range(start=df.index[-1], periods=forecast_steps+1, closed='right')
plt.plot(forecast_index, final_forecast, marker='o', linestyle='--', color='red', label='Final Forecast')
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("ARIMAX + AutoML (TPOT) Forecast")
plt.legend()
plt.show()


In [ ]:
import itertools
import warnings
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from tpot import TPOTRegressor  # AutoML tool for regression

warnings.filterwarnings("ignore")

# ----------------------------
# 1. Load and Prepare Data
# ----------------------------
# Assume your CSV file has columns: Date, Open, High, Low, Close, Volume, Turnover
df = pd.read_csv('your_stock_data.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

# ----------------------------
# 2. (Optional) Stationarity Check
# ----------------------------
# Here you might want to test for stationarity (e.g., using the ADF test) and difference the series if needed.
# For brevity, this example assumes that either the series is stationary or that differencing is handled within the order search.

# ----------------------------
# 3. Grid Search for the Best ARIMAX Order Based on BIC
# ----------------------------
# Define the range for p, d, and q.
p_range = range(0, 3)  # Adjust as needed
d_range = range(0, 2)  # Adjust as needed
q_range = range(0, 3)  # Adjust as needed

# Create all possible combinations of p, d, q
pdq = list(itertools.product(p_range, d_range, q_range))

# Define the exogenous variables
exog_vars = ['Open', 'High', 'Low', 'Volume', 'Turnover']
exog = df[exog_vars]

lowest_bic = np.inf
best_order = None
results_dict = {}

print("Searching for the best ARIMAX order based on BIC...")
for order in pdq:
    try:
        # Fit ARIMAX model with given order and exogenous regressors
        model = ARIMA(endog=df['Close'], exog=exog, order=order)
        results = model.fit()
        bic = results.bic
        results_dict[order] = bic
        print(f"Order {order} BIC: {bic:.2f}")
        if bic < lowest_bic:
            lowest_bic = bic
            best_order = order
    except Exception as e:
        # Skip non-converging models
        print(f"Order {order} failed: {e}")
        continue

print("\nBest ARIMAX order based on BIC:", best_order, "with BIC:", lowest_bic)

# ----------------------------
# 4. Fit ARIMAX with the Best Order and Forecast Next 6 Days
# ----------------------------
forecast_steps = 6

# In practice, forecast exogenous values for the next period.
# For demonstration, assume the last observed exogenous values persist.
last_exog = exog.iloc[-1].values.reshape(1, -1)
exog_forecast = pd.DataFrame(np.repeat(last_exog, forecast_steps, axis=0), 
                             columns=exog.columns)

# Fit the ARIMAX model using the best order.
best_model = ARIMA(endog=df['Close'], exog=exog, order=best_order)
best_model_fit = best_model.fit()
arimax_forecast = best_model_fit.forecast(steps=forecast_steps, exog=exog_forecast)
print("\nARIMAX Forecast (Linear Component):")
print(arimax_forecast)

# Compute in-sample fitted values and ARIMAX residuals.
df['ARIMAX_Fitted'] = best_model_fit.fittedvalues
df['ARIMAX_Residuals'] = df['Close'] - df['ARIMAX_Fitted']

# ----------------------------
# 5. Use AutoML (TPOT) to Forecast ARIMAX Residuals
# ----------------------------
# Prepare training data for residual prediction using the same exogenous variables.
X_res = exog.copy()
y_res = df['ARIMAX_Residuals']

# Drop missing values if any.
X_res = X_res.dropna()
y_res = y_res.loc[X_res.index]

# Configure TPOT for residual prediction.
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2, random_state=42)
tpot.fit(X_res, y_res)

# Forecast ARIMAX residuals for the next 6 days.
ml_residual_forecast = tpot.predict(exog_forecast)
print("\nTPOT Forecast for ARIMAX Residuals:")
print(ml_residual_forecast)

# ----------------------------
# 6. Combine ARIMAX and AutoML Forecasts
# ----------------------------
final_forecast = arimax_forecast + ml_residual_forecast
print("\nFinal Combined Forecast (ARIMAX + ML Residual):")
print(final_forecast)

# ----------------------------
# 7. Visualization
# ----------------------------
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label='Historical Close Price')
plt.plot(df.index, df['ARIMAX_Fitted'], label='ARIMAX Fitted', alpha=0.7)
# Generate forecast dates – adjust frequency as needed.
forecast_index = pd.date_range(start=df.index[-1], periods=forecast_steps+1, closed='right')
plt.plot(forecast_index, final_forecast, marker='o', linestyle='--', color='red', label='Final Forecast')
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("ARIMAX + AutoML (TPOT) Forecast with Best Order (Based on BIC)")
plt.legend()
plt.show()
